In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../../enfify")

from glob import glob

import yaml
from analyses import (classification_rodriguez, eer_feature_threshold,
                      feature_zscore)
from feature_processing import get_hilbert_phase
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from utils import read_wavfile

In [3]:
with open("../../enfify/defaults.yml", "r") as f:
    config = yaml.safe_load(f)
config["downsample"]["is_enabled"] = False

In [4]:
authentic_paths = sorted(glob("../../data/raw/synthetic/uncut/train_audio/*.wav"))
tampered_paths = sorted(glob("../../data/raw/synthetic/cut/train_cut_audio/*.wav"))

In [5]:
labels = []
features = []

_accuracys = []
_gammas = []
for auth_path, tamp_path in zip((pbar:=tqdm(authentic_paths)), tampered_paths):
    for label, path in zip([True, False], [auth_path, tamp_path]):
        sig, fs = read_wavfile(path)
        x, phase = get_hilbert_phase(sig, fs, config)

        feature = feature_zscore(phase)
        features.append(feature)
        labels.append(label)

    gamma = eer_feature_threshold(labels, features)
    results = [feature < gamma for feature in features]
    accuracy = accuracy_score(labels, results)
    pbar.set_postfix_str(f"Accuracy: {accuracy*100:.2f} %, gamma: {gamma:.2f}")
    _accuracys.append(accuracy)
    _gammas.append(gamma)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [33:58<00:00,  2.04s/it, Accuracy: 86.65 %, gamma: 2.62] 
